In [2]:
import pandas as pd
import xgboost as xgb
import numpy as np

In [3]:
data_23 = pd.read_csv('data/laliga22-23.csv')
data_24 = pd.read_csv('data/laliga23-24.csv')
data_25 = pd.read_csv('data/laliga24-25.csv')

In [4]:
data_23.describe()

,TeamID,GoalsForHome,GoalsAgainstHome,GoalsForAway,GoalsAgainstAway,WinsHome,DrawsHome,LossesHome,WinsAway,DrawsAway,...,Losses,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Rank,last_5_winners,Rating
count,20.00000,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,2.000000e+01,20.000000,20.00000,20.000000,20.000000
mean,10.50000,27.65000,20.100000,20.100000,27.650000,9.100000,4.450000,5.450000,5.450000,4.450000,...,14.550000,52.550000,1.383500,49.430000,49.425000,1.776357e-16,-0.000500,10.50000,0.250000,6.809500
std,5.91608,7.30735,6.576673,6.679899,7.292931,2.971797,1.503505,2.282081,3.332061,1.700619,...,4.406634,15.882712,0.418082,11.671562,10.536047,1.973679e+01,0.519397,5.91608,0.638666,0.095227
min,1.00000,18.00000,4.000000,11.000000,16.000000,3.000000,1.000000,1.000000,1.000000,1.000000,...,6.000000,25.000000,0.660000,35.200000,33.000000,-3.260000e+01,-0.860000,1.00000,0.000000,6.690000
25%,5.75000,22.00000,16.000000,14.500000,21.750000,7.000000,3.000000,4.000000,3.000000,3.750000,...,12.000000,42.000000,1.110000,41.075000,42.900000,-1.127500e+01,-0.300000,5.75000,0.000000,6.747500
50%,10.50000,26.00000,20.500000,20.000000,28.500000,9.000000,5.000000,6.000000,4.000000,4.500000,...,15.000000,49.000000,1.290000,47.300000,48.150000,-5.100000e+00,-0.130000,10.50000,0.000000,6.780000
75%,15.25000,30.50000,23.500000,25.000000,31.250000,11.000000,5.250000,7.000000,7.250000,6.000000,...,17.000000,61.000000,1.605000,53.225000,56.125000,1.317500e+01,0.347500,15.25000,0.000000,6.822500
max,20.00000,44.00000,35.000000,33.000000,43.000000,15.000000,7.000000,10.000000,13.000000,7.000000,...,23.000000,88.000000,2.320000,75.500000,70.000000,4.230000e+01,1.110000,20.00000,2.000000,7.040000


In [5]:
data_24.describe()

,TeamID,GoalsForHome,GoalsAgainstHome,GoalsForAway,GoalsAgainstAway,WinsHome,DrawsHome,LossesHome,WinsAway,DrawsAway,...,Losses,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Rank,last_5_winners,Rating
count,20.00000,20.00000,20.000000,20.00000,20.000000,20.000000,20.000000,20.000000,20.0000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000
mean,10.50000,28.20000,22.050000,22.05000,28.200000,8.350000,5.350000,5.300000,5.3000,5.350000,...,13.650000,51.650000,1.358500,49.555000,49.550000,-0.005000,-0.000500,10.50000,0.250000,6.886500
std,5.91608,11.41375,6.402919,8.10117,7.395589,4.307582,2.254236,2.735729,3.4504,1.531253,...,5.869412,20.368384,0.536747,11.878839,7.961784,16.500127,0.433583,5.91608,0.638666,0.118067
min,1.00000,16.00000,9.000000,10.00000,17.000000,1.000000,1.000000,0.000000,0.0000,3.000000,...,1.000000,21.000000,0.550000,33.200000,35.400000,-23.500000,-0.620000,1.00000,0.000000,6.720000
25%,5.75000,20.00000,19.000000,15.50000,22.750000,6.000000,4.750000,3.000000,2.7500,4.000000,...,9.750000,40.000000,1.050000,42.525000,42.575000,-9.800000,-0.260000,5.75000,0.000000,6.807500
50%,10.50000,23.50000,20.500000,21.00000,27.000000,7.500000,6.000000,5.500000,4.5000,5.500000,...,15.000000,45.500000,1.195000,45.300000,50.050000,-3.500000,-0.090000,10.50000,0.000000,6.860000
75%,15.25000,37.50000,26.000000,26.50000,32.250000,9.750000,6.250000,7.000000,7.2500,6.000000,...,17.000000,62.000000,1.632500,53.425000,54.925000,5.550000,0.150000,15.25000,0.000000,6.930000
max,20.00000,53.00000,36.000000,39.00000,47.000000,16.000000,9.000000,10.000000,13.0000,8.000000,...,25.000000,95.000000,2.500000,77.600000,64.200000,35.900000,0.940000,20.00000,2.000000,7.200000


In [6]:
data_25.describe()

,TeamID,GoalsForHome,GoalsAgainstHome,GoalsForAway,GoalsAgainstAway,WinsHome,DrawsHome,LossesHome,WinsAway,DrawsAway,...,Losses,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Rank,last_5_winners,Rating
count,20.00000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,...,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.00000,20.000000
mean,10.50000,27.600000,22.150000,22.15000,27.600000,8.450000,4.850000,5.700000,5.700000,4.850000,...,14.150000,52.150000,1.373000,48.910000,48.910000,-0.010000,0.000500,10.50000,0.25000,6.879000
std,5.91608,11.189281,4.987089,8.41224,9.511212,3.471311,1.980829,2.939746,3.357631,2.497894,...,6.251526,17.898177,0.471125,15.049179,8.535616,21.332182,0.561647,5.91608,0.71635,0.116117
min,1.00000,11.000000,13.000000,15.00000,15.000000,3.000000,1.000000,1.000000,1.000000,0.000000,...,6.000000,16.000000,0.420000,34.400000,33.400000,-33.000000,-0.870000,1.00000,0.00000,6.690000
25%,5.75000,20.000000,19.750000,16.75000,19.750000,6.000000,3.750000,3.000000,3.750000,3.000000,...,9.500000,41.750000,1.102500,38.300000,43.775000,-9.475000,-0.250000,5.75000,0.00000,6.817500
50%,10.50000,25.000000,21.000000,19.00000,29.000000,8.000000,5.500000,5.500000,5.000000,5.000000,...,15.500000,47.000000,1.235000,43.050000,47.200000,-4.450000,-0.115000,10.50000,0.00000,6.855000
75%,15.25000,32.250000,26.250000,25.25000,31.250000,10.250000,6.000000,8.000000,7.250000,6.000000,...,18.000000,62.500000,1.645000,54.325000,52.350000,11.975000,0.317500,15.25000,0.00000,6.892500
max,20.00000,52.000000,32.000000,50.00000,58.000000,16.000000,8.000000,12.000000,14.000000,10.000000,...,30.000000,88.000000,2.320000,91.500000,67.600000,49.500000,1.300000,20.00000,3.00000,7.180000


In [7]:
data_23 = data_23.drop(columns=['TeamID'])
data_24 = data_24.drop(columns=['TeamID'])
data_25 = data_25.drop(columns=['TeamID'])

data = pd.concat([data_24, data_25,data_23], ignore_index=True)

data.describe()

,GoalsForHome,GoalsAgainstHome,GoalsForAway,GoalsAgainstAway,WinsHome,DrawsHome,LossesHome,WinsAway,DrawsAway,LossesAway,...,Losses,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Rank,last_5_winners,Rating
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.000000,60.000000,60.000000,60.00000,60.000000,60.000000,60.000000,60.000000
mean,27.816667,21.433333,21.433333,27.816667,8.633333,4.883333,5.483333,5.483333,4.883333,8.633333,...,14.116667,52.116667,1.371667,49.298333,49.295000,-0.00500,-0.000167,10.500000,0.250000,6.858333
std,9.977078,6.003859,7.694080,7.996804,3.579445,1.940572,2.626409,3.326787,1.957962,3.283067,...,5.483694,17.836443,0.469735,12.740613,8.927627,18.96486,0.499001,5.814943,0.654191,0.113901
min,11.000000,4.000000,10.000000,15.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,16.000000,0.420000,33.200000,33.000000,-33.00000,-0.870000,1.000000,0.000000,6.690000
25%,20.750000,18.750000,15.000000,21.750000,6.000000,3.000000,3.000000,3.000000,4.000000,7.000000,...,10.000000,41.000000,1.080000,41.250000,43.275000,-10.02500,-0.260000,5.750000,0.000000,6.780000
50%,26.000000,21.000000,20.500000,28.000000,8.000000,5.000000,6.000000,4.000000,5.000000,9.000000,...,15.000000,48.500000,1.275000,45.150000,48.300000,-4.25000,-0.110000,10.500000,0.000000,6.825000
75%,33.500000,25.250000,25.000000,32.000000,11.000000,6.000000,7.000000,7.250000,6.000000,10.000000,...,17.000000,61.000000,1.605000,54.200000,54.375000,11.50000,0.302500,15.250000,0.000000,6.910000
max,53.000000,36.000000,50.000000,58.000000,16.000000,9.000000,12.000000,14.000000,10.000000,18.000000,...,30.000000,95.000000,2.500000,91.500000,70.000000,49.50000,1.300000,20.000000,3.000000,7.200000


In [8]:
teams = data['Team'].unique()
teams_dict = {team: i+1 for i, team in enumerate(teams)}
teams_dict

{'Almeria': 1,
 'Sevilla': 2,
 'Sociedad': 3,
 'Las Palmas': 4,
 'Ath Bilbao': 5,
 'Celta': 6,
 'Villarreal': 7,
 'Getafe': 8,
 'Cadiz': 9,
 'Ath Madrid': 10,
 'Mallorca': 11,
 'Valencia': 12,
 'Osasuna': 13,
 'Girona': 14,
 'Barcelona': 15,
 'Betis': 16,
 'Alaves': 17,
 'Granada': 18,
 'Vallecano': 19,
 'Real Madrid': 20,
 'Valladolid': 21,
 'Espanol': 22,
 'Leganes': 23,
 'Elche': 24}

In [9]:
data['Team'] = data['Team'].map(teams_dict)

In [10]:
# Concatenate training data from 23 and 24 seasons
X_train = pd.concat([data_23, data_24], ignore_index=True)
X_train = X_train.drop(columns=['Rank'])

# Prepare test data from 25 season (drop target column)
X_test = data_25.drop(columns=['Rank'])

# Concatenate training labels from 23 and 24 seasons
y_train = pd.concat([data_23["Rank"], data_24["Rank"]], ignore_index=True)

# Test labels from 25 season
y_test = data_25["Rank"]


In [11]:
X_train["Team"] = X_train["Team"].map(teams_dict)
X_test["Team"] = X_test["Team"].map(teams_dict)

In [12]:
pos = sum(y_train == 1)
neg = sum(y_train == 0)
scale_pos_weight = neg / pos

model = xgb.XGBRegressor(
    max_depth=4,
    n_estimators=200,
    random_state=42
)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)

In [13]:
predicted_ranks = model.predict(X_test)

In [14]:
# Invert ranks to "scores" (higher score = better chance)
inv_ranks = 1 / predicted_ranks

# Normalize to get probabilities summing to 1
win_probs = inv_ranks / np.sum(inv_ranks)

# Add to test dataframe for review
result = data_25.copy()
result['LaLiga Champions'] = win_probs *100
ranks = np.arange(1,21)
result = result.sort_values(by='LaLiga Champions', ascending=False)
result['LaLiga Champions'] = result['LaLiga Champions'].round(2).astype(str) + '%'
result['Predicted_Rank'] = ranks

print(result[['Team','LaLiga Champions', 'Predicted_Rank']])

           Team LaLiga Champions  Predicted_Rank
11    Barcelona           16.78%               1
14  Real Madrid           11.93%               2
17   Ath Madrid           11.64%               3
9    Villarreal            8.89%               4
0    Ath Bilbao            6.44%               5
1         Betis            5.44%               6
2         Celta            4.55%               7
18    Vallecano            3.87%               8
4       Osasuna            3.44%               9
7      Mallorca            3.39%              10
6      Sociedad            3.29%              11
5      Valencia            2.83%              12
16       Alaves            2.53%              13
10      Sevilla            2.35%              14
13       Getafe            2.35%              15
12      Espanol            2.26%              16
15      Leganes            2.14%              17
19       Girona            2.11%              18
3    Las Palmas            1.95%              19
8    Valladolid     